In [1]:
import pandas as pd
import time
from datetime import datetime, timezone
from sqlalchemy import create_engine, Column, Float, String, Integer, DateTime
from sqlalchemy.orm import sessionmaker, declarative_base
from polygon import RESTClient
import pymongo

In [2]:
polygon_api = 'beBybSi8daPgsTp5yx5cHtHpYcrjp5Jq'
sql_database_path = 'fx_rates_sql.db'
csv_output_path = 'fx_rates.csv'
mongo_db_url = "mongodb://localhost:27017/"
mongo_db_name = "fx_rates_mg"
mongo_collection_name = "fx"
fx_list = [("EUR", "USD"), ("GBP", "USD"), ("USD", "CHF")]

In [3]:
polygon = RESTClient(polygon_api)

In [10]:
Base = declarative_base()

class fx_data(Base):
    __tablename__ = 'fx_rates'
    id = Column(Integer, primary_key= True)
    pair = Column(String)
    fx_timestamp = Column(DateTime)
    rate = Column(Float)
    entry_timestamp = Column(DateTime, default=datetime.now(timezone.utc))

    def __init__(self, pair, rate, fx_timestamp, entry_timestamp):
        self.pair = pair
        self.rate = rate
        self.fx_timestamp = fx_timestamp
        self.entry_timestamp = entry_timestamp

engine = create_engine('sqlite:///{}'.format(sql_database_path))
Base.metadata.create_all(engine)

Session = sessionmaker(bind= engine)
session = Session()

In [5]:
mongo = pymongo.MongoClient(mongo_db_url)
mongo_db = mongo[mongo_db_name]
mongo_collection = mongo_db[mongo_collection_name]

In [ ]:
start_time = time.time()
log_time = time.time()

while((time.time() - start_time) < 7200):
    iteration_start_time = time.time()

    for base_c, quote_c in fx_list:
        raw_data = polygon.get_real_time_currency_conversion(base_c,quote_c)
        fx_timestamp = datetime.fromtimestamp(raw_data.last.timestamp / 1000,tz= timezone.utc)
        rate = raw_data.converted
        pair = '{},{}'.format(base_c,quote_c)
        
        if pair is None:
            print('failed to fetch data '+str(datetime.now()))
            break

        fx_sql_data = fx_data(pair,rate,fx_timestamp,datetime.now(timezone.utc))
        session.add(fx_sql_data)

        fx_mongo_data = {'pair' : pair, 'fx_timestamp' : fx_timestamp, 'rate' : rate, 'entry_timestamp' : datetime.now(tz=timezone.utc)}
        mongo_collection.insert_one(fx_mongo_data)
    
    try:
        session.commit()
    except Exception as e:
        session.rollback()
        print(f"An error occurred: {e}")

    iteration_end_time = time.time()

    if(iteration_end_time - iteration_start_time) < 1:
        time.sleep(1-(iteration_end_time - iteration_start_time))
    
    if((iteration_end_time - log_time)>60):
        log_time = time.time()
        print(datetime.fromtimestamp(log_time))

session.close()

In [ ]:
# mongo_collection.delete_many({})
# documents = mongo_collection.find()
# for doc in documents:
#     print(doc)

In [ ]:
# session.query(fx_data).delete()
# session.commit()
#pd.read_sql('fx_rates', engine)